## Measuring VNC length

To measure the VNC length, we need to go over these steps:

1. Calculate an ROI for each image slice
2. Measure the maximum feret diameter of each slice

> The maximum feret diameter is computed as the longest distance between points around a region's convex hull contour.

The measurement is done using the structural channel, so that the pixel intensity changes less over time, making the ROI easier to calculate.
There is a lot of fluctuation in the lengths, because the embryo flickers over time, causing the VNC to change position and deform.
Part of the change in the VNC length is that the ROI sometimes changes and accounts for regions that are actually part of the brain lobes.

To filter out the effects of movement and deformation, a baseline with a first degree polynomial was plotted with the image.

Because we are using the ROI to calculate the VNC length, the more accurate our ROI is, the better the results will be.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os
from pathlib import Path

import numpy as np
from tifffile import imread
import matplotlib.pyplot as plt

from pasnascope import roi, vnc_length

In [ ]:
project_dir = Path(os.getcwd()).parent
img_dir = os.path.join(project_dir, 'data', 'embryos')

# All structural channel movies end with the suffix ch2
filenames = [f for f in sorted(os.listdir(img_dir)) if f.endswith('ch2.tif')]

In [ ]:
i = 0
PIXEL_WIDTH = 1.62
img = imread(os.path.join(img_dir , filenames[i]), key=range(0,1000))

img_roi = roi.get_roi(img, window=10)
vnc_lengths = vnc_length.measure_VNC(img_roi)*PIXEL_WIDTH

reg = vnc_length.fit_regression(vnc_lengths)

initial_length = np.average(vnc_lengths[:5])
final_length = np.average(vnc_lengths[-5:])

x = np.arange(vnc_lengths.size)
fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
ax.plot(vnc_lengths)
ax.plot(x, reg(x), 'k--');

### Effect of sampling when measuring VNC length

Different intervals where selected to check how they affect the change in the VNC length.
The idea is to also see if the measurements taken manually can be directly compared with the values in here. 

In [ ]:
i = 0
PIXEL_WIDTH = 1.62
img = imread(os.path.join(img_dir , filenames[i]), key=range(0,1000))
print(filenames[i])

img_roi = roi.get_roi(img, window=1)
vnc_lengths = vnc_length.measure_VNC(img_roi)*PIXEL_WIDTH

reg = vnc_length.fit_regression(vnc_lengths)

windows = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
deltas = []
for w in windows:
    vnc_samples = vnc_lengths[::w]
    reg = vnc_length.fit_regression(vnc_samples)
    deltas.append((reg(vnc_samples.size-1)-reg(0))/reg(0))

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle("% condensation x sampling interval")
ax.plot(windows, deltas, 'r:o');